Goal: To predict The Salary Based on some of the Factors of the Job Position

In [4]:
import glassdoor_scraper as gs


import pandas as pd

import sys

import re

import matplotlib.pyplot as plt

import plotly.express as px

import numpy as np

import scipy

import sklearn

import seaborn

from datetime import datetime, timedelta

from matplotlib import dates as mpl_dates

from matplotlib.animation import FuncAnimation

1. Scrapping Data

In [5]:
path = "C:/Users/enriq/Anaconda3/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe"

In [6]:
# df=gs.get_jobs("data scientist", 12, False, path, 15) # didnt work for me 

In [7]:
df = pd.read_csv("C:/Users/enriq/Your_env_name Notebooks/csv glassdoor/glassdoor_datascience_US.csv", engine="python", encoding='utf-8', error_bad_lines=False)

Skipping line 753: unexpected end of data


2. Data cleaning

In [8]:
pd.set_option("display.max_rows", 85)

In [9]:
df.dtypes

Unnamed: 0             int64
Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Headquarters          object
Size                  object
Founded                int64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
Competitors           object
dtype: object

In [10]:
df.columns = df.columns.str.lower() # Making title columns lower case

In [11]:
filt = df["salary estimate"].str.contains("Per Hour")

In [12]:
df["salary estimate"].loc[filt]

197             $17-$24 Per Hour(Glassdoor est.)
209             $21-$34 Per Hour(Glassdoor est.)
240             $18-$25 Per Hour(Glassdoor est.)
247             $21-$34 Per Hour(Glassdoor est.)
257             $15-$25 Per Hour(Glassdoor est.)
307             $17-$24 Per Hour(Glassdoor est.)
337             $21-$34 Per Hour(Glassdoor est.)
427             $18-$25 Per Hour(Glassdoor est.)
437             $24-$39 Per Hour(Glassdoor est.)
456             $21-$34 Per Hour(Glassdoor est.)
464    Employer Provided Salary:$25-$28 Per Hour
522             $21-$29 Per Hour(Glassdoor est.)
523             $10-$17 Per Hour(Glassdoor est.)
632             $18-$25 Per Hour(Glassdoor est.)
652             $24-$39 Per Hour(Glassdoor est.)
682             $21-$34 Per Hour(Glassdoor est.)
694    Employer Provided Salary:$25-$28 Per Hour
Name: salary estimate, dtype: object

In [13]:
df["hourly"] = filt

In [14]:
pattern = re.compile(r"[\d{2}]+")

In [15]:
lista = [] # salary parsing
for i in df["salary estimate"]:
    x = pattern.findall(i)
    try:
        x[0] = int(x[0])
        x[1] = int(x[1])
    except IndexError:
        pass
    x=pd.Series(x).mean()
    lista.append(x)


In [16]:
df["salary estimate"] = pd.Series(lista)

In [17]:
df["salary estimate"].sample(50)

109     56.5
458     99.5
433    154.5
349     96.0
554    153.0
117    237.5
699     80.0
638     95.0
329      1.0
487    112.5
445      1.0
515      1.0
680      1.0
575    113.5
55     114.5
563     93.5
630     48.5
222      1.0
298      1.0
725     94.0
108     99.0
291    100.0
102     85.0
243    124.0
244      1.0
340      1.0
438      1.0
382    110.0
385      1.0
164    105.0
294      1.0
402      1.0
1       87.5
697     70.0
59      75.5
582     80.5
171    137.5
726     87.5
441      1.0
674    139.5
667      1.0
241      1.0
642    111.5
490     51.0
600     71.0
242     52.5
207     98.5
368     81.0
558    128.0
315      1.0
Name: salary estimate, dtype: float64

In [18]:
df["salary estimate"].loc[filt]

197    20.5
209    27.5
240    21.5
247    27.5
257    20.0
307    20.5
337    27.5
427    21.5
437    31.5
456    27.5
464    26.5
522    25.0
523    13.5
632    21.5
652    31.5
682    27.5
694    26.5
Name: salary estimate, dtype: float64

In [19]:
df.isna().sum()

unnamed: 0           0
job title            0
salary estimate      0
job description      0
rating               0
company name         0
location             0
headquarters         0
size                 0
founded              0
type of ownership    0
industry             0
sector               0
revenue              0
competitors          0
hourly               0
dtype: int64

In [20]:
df_splitted = df["location"].str.split(",", expand=True) # city/state columns

In [21]:
df = pd.concat([df, df_splitted], axis=1)

In [22]:
df

,unnamed: 0,job title,salary estimate,job description,rating,company name,location,headquarters,size,founded,type of ownership,industry,sector,revenue,competitors,hourly,0,1,2
0,0,Data Scientist,72.0,"Data Scientist\r\nLocation: Albuquerque, NM\r\...",3.8,Tecolote Research\r\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,False,Albuquerque,NM,None
1,1,Healthcare Data Scientist,87.5,What You Will Do:\r\n\r\nI. General Summary\r\...,3.4,University of Maryland Medical System\r\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,False,Linthicum,MD,None
2,2,Data Scientist,85.0,"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\r\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,False,Clearwater,FL,None
3,3,Data Scientist,76.5,*Organization and Job ID**\r\nJob ID: 310709\r...,3.8,PNNL\r\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",False,Richland,WA,None
4,4,Data Scientist,114.5,Data Scientist\r\nAffinity Solutions / Marketi...,2.9,Affinity Solutions\r\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",False,New York,NY,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,746,"Technology-Minded, Data Professional Opportuni...",70.5,"At VU, we help Veterans buy homes. To make tha...",4.7,Veterans United Home Loans\r\n4.7,"Columbia, MO","Columbia, MO",1001 to 5000 employees,2002,Company - Private,Lending,Finance,Unknown / Non-Applicable,-1,False,Columbia,MO,None
747,747,Big Data Engineer,118.5,Overview\r\n\r\n\r\nTechnology is constantly c...,3.3,Peraton\r\n3.3,"Chantilly, VA","Herndon, VA",1001 to 5000 employees,2017,Company - Private,Aerospace & Defense,Aerospace & Defense,$1 to $2 billion (USD),-1,False,Chantilly,VA,None
748,748,Salesforce Analytics Consultant,66.5,"Wave6, an Emtec company, innovates with cuttin...",3.9,"Emtec, Inc.\r\n3.9","Chicago, IL","Jacksonville, FL",501 to 1000 employees,1995,Company - Private,Enterprise Software & Network Solutions,Information Technology,$100 to $500 million (USD),-1,False,Chicago,IL,None
749,749,Managing Data Scientist/ML Engineer,107.5,Managing Data Scientist/ML Engineer\r\n\r\nApp...,3.4,PA Consulting\r\n3.4,"Boston, MA","London, United Kingdom",1001 to 5000 employees,1943,Company - Private,Consulting,Business Services,$100 to $500 million (USD),"McKinsey & Company, Accenture, Deloitte",False,Boston,MA,None


In [23]:
df = df.drop(["location",2], axis=1)

In [24]:
df.rename(columns={0:"city", 1: "state"}, inplace=True)

In [25]:
df.tail(80)

,unnamed: 0,job title,salary estimate,job description,rating,company name,headquarters,size,founded,type of ownership,industry,sector,revenue,competitors,hourly,city,state
671,671,Food Scientist - Developer,54.0,Palermo Villa Inc. is interested in a high-ene...,3.3,Palermo's Pizza\r\n3.3,"Milwaukee, WI",501 to 1000 employees,1964,Company - Private,Food & Beverage Manufacturing,Manufacturing,Unknown / Non-Applicable,-1,False,Milwaukee,WI
672,672,Senior Data Engineer,109.0,Sr. Data Engineer FTE for Franklin\r\n\r\nTher...,3.2,Equian LLC\r\n3.2,"Indianapolis, IN",1001 to 5000 employees,2004,Company - Private,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable,-1,False,Franklin,TN
673,673,Data Engineer,109.0,MetroStar Systems is looking for a Data Engine...,3.4,MetroStar Systems\r\n3.4,"Reston, VA",201 to 500 employees,1999,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),-1,False,Rockville,MD
674,674,Senior Data Scientist,139.5,"The Infrastructure, Architecture, and Tools te...",4.4,MathWorks\r\n4.4,"Natick, MA",1001 to 5000 employees,1984,Company - Private,Computer Hardware & Software,Information Technology,$1 to $2 billion (USD),-1,False,Natick,MA
675,675,"Director II, Data Science - GRM Actuarial",254.0,Advance your career at Liberty Mutual Insuranc...,3.3,Liberty Mutual Insurance\r\n3.3,"Boston, MA",10000+ employees,1912,Company - Private,Insurance Carriers,Insurance,$10+ billion (USD),"Travelers, Allstate, State Farm",False,Chicago,IL
676,676,Machine Learning Engineer,125.0,We are seeking a highly-experienced ML Enginee...,3.2,Information Builders\r\n3.2,"New York, NY",1001 to 5000 employees,1975,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,"Qlik, Tableau Software, Informatica",False,New York,NY
677,677,Data Engineer,1.0,Our client is a leading hedge fund looking to ...,4.1,NJF Global Holdings\r\n4.1,"London, United Kingdom",51 to 200 employees,2003,Company - Private,Staffing & Outsourcing,Business Services,$10 to $25 million (USD),-1,False,New York,NY
678,678,"Sr Expert Data Science, Advanced Visual Analyt...",106.5,"Posting Title\r\nSr Expert Data Science, Advan...",3.8,Novartis\r\n3.8,"Basel, Switzerland",10000+ employees,1996,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),-1,False,Cambridge,MA
679,679,"Scientist, Molecular/Cellular Biologist",73.0,"[1]Scientist, Molecular/Cellular Biologist\r\n...",2.9,Audentes Therapeutics\r\n2.9,"San Francisco, CA",201 to 500 employees,2012,Subsidiary or Business Segment,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,"BioMarin Pharmaceutical, Sangamo Therapeutics,...",False,South San Francisco,CA
680,680,Clinical Data Manager,1.0,"At Alector, our mission is to develop therapie...",5.0,Alector\r\n5.0,"South San Francisco, CA",51 to 200 employees,2013,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,-1,False,South San Francisco,CA


In [26]:
pattern2 = re.compile(r"\d*\.?\d+") # pattern to parse a decimal number from within a string

In [27]:
pattern2.findall(df["company name"][0])

['3.8']

In [28]:
lista2=[]
for i in df["company name"]:
    x = pattern2.findall(i)
    lista2.append(x)


In [29]:
lista2

[['3.8'],
 ['3.4'],
 ['4', '4.8'],
 ['3.8'],
 ['2.9'],
 ['3.4'],
 ['4.1'],
 ['20', '20', '3.8'],
 ['3.3'],
 ['4.6'],
 ['3.5'],
 ['4.1'],
 ['3.2'],
 ['4.1'],
 ['3.7'],
 ['3.6'],
 ['3.9'],
 ['2', '4.3'],
 ['4.2'],
 ['4.0'],
 ['3.2'],
 ['3.9'],
 ['3.8'],
 ['3', '4.3'],
 ['3.9'],
 ['4.0'],
 ['4.0'],
 ['3.5'],
 ['3.7'],
 ['4.0'],
 ['4', '4.8'],
 ['3.8'],
 ['3.6'],
 ['3.8'],
 ['3.8'],
 ['4.7'],
 ['4.2'],
 ['3.5'],
 ['4.7'],
 ['3.5'],
 ['3.5'],
 ['4.2'],
 ['4.1'],
 ['3.6'],
 ['4.2'],
 ['3.4'],
 ['4.3'],
 ['5.0'],
 ['5.0'],
 ['3.7'],
 ['3.1'],
 ['4.7'],
 ['3.7'],
 ['4.5'],
 ['3.8'],
 ['3.3'],
 ['4.6'],
 ['3.7'],
 ['3.7'],
 ['3.5'],
 ['4.1'],
 ['1904', '4.7'],
 ['4.1'],
 ['3.4'],
 ['3.1'],
 ['4.4'],
 ['4.3'],
 ['3.8'],
 ['3.8'],
 ['4.5'],
 ['3.8'],
 ['2.2'],
 ['2.8'],
 ['4.7'],
 ['4.9'],
 ['3.1'],
 ['3.2'],
 ['4.0'],
 ['4.4'],
 ['3.6'],
 ['2.7'],
 ['3.9'],
 ['3.1'],
 ['4.0'],
 ['2', '4.3'],
 ['1.9'],
 ['3.3'],
 ['4.4'],
 ['3.9'],
 ['4.7'],
 ['4.5'],
 ['4.3'],
 ['4.0'],
 ['4.4'],
 ['3.2'],
 ['20

In [30]:
lista2 = pd.Series(lista2)

In [31]:
lista2 = lista2.astype(str).str.replace("]","").str.replace("[","").str.replace("'","")

In [32]:
lista2

0         3.8
1         3.4
2      4, 4.8
3         3.8
4         2.9
        ...  
746       4.7
747       3.3
748       3.9
749       3.4
750       4.7
Length: 751, dtype: object

In [33]:
lista3 = [] # parsing decimal number from company column
for i in lista2:
    if len(i)==3:
        i = float(i)
        lista3.append(i)
    else:
        lista3.append(i)

In [34]:
lista3 = pd.Series(lista3)

In [35]:
df["parsed_num"] = lista3

In [36]:
df["founded"]

0      1973
1      1984
2      2010
3      1965
4      1998
       ... 
746    2002
747    2017
748    1995
749    1943
750    2010
Name: founded, Length: 751, dtype: int64

In [37]:
lista3 = [] # changing to datetime if format allows it. Otherwise, dont change anything
for i in range(len(df["founded"])):
    try:
        x=pd.to_datetime(df["founded"][i], format="%Y")
        lista3.append(x)
    except ValueError:
        lista3.append(df["founded"][i])

In [38]:
lista4 = [] # we had to deal with few -1 values
for i in lista3:
    if i != -1:
        x=round((datetime.now()-i).days/365)
        lista4.append(x)
    else:
        lista4.append(i)

In [39]:
lista4=pd.Series(lista4)

In [40]:
df["years_old"] = lista4

In [41]:
df["years_old"].replace(-1,np.nan, inplace=True) # converting those -1 values into nan

In [42]:
df["years_old"].isna().sum()

70

In [43]:
df["salary estimate"].replace(1, np.nan, inplace=True)

In [44]:
df["salary estimate"].isna().sum()

160

In [45]:
df

,unnamed: 0,job title,salary estimate,job description,rating,company name,headquarters,size,founded,type of ownership,industry,sector,revenue,competitors,hourly,city,state,parsed_num,years_old
0,0,Data Scientist,72.0,"Data Scientist\r\nLocation: Albuquerque, NM\r\...",3.8,Tecolote Research\r\n3.8,"Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,False,Albuquerque,NM,3.8,47.0
1,1,Healthcare Data Scientist,87.5,What You Will Do:\r\n\r\nI. General Summary\r\...,3.4,University of Maryland Medical System\r\n3.4,"Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,False,Linthicum,MD,3.4,36.0
2,2,Data Scientist,85.0,"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\r\n4.8,"Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,False,Clearwater,FL,"4, 4.8",10.0
3,3,Data Scientist,76.5,*Organization and Job ID**\r\nJob ID: 310709\r...,3.8,PNNL\r\n3.8,"Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",False,Richland,WA,3.8,55.0
4,4,Data Scientist,114.5,Data Scientist\r\nAffinity Solutions / Marketi...,2.9,Affinity Solutions\r\n2.9,"New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",False,New York,NY,2.9,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,746,"Technology-Minded, Data Professional Opportuni...",70.5,"At VU, we help Veterans buy homes. To make tha...",4.7,Veterans United Home Loans\r\n4.7,"Columbia, MO",1001 to 5000 employees,2002,Company - Private,Lending,Finance,Unknown / Non-Applicable,-1,False,Columbia,MO,4.7,18.0
747,747,Big Data Engineer,118.5,Overview\r\n\r\n\r\nTechnology is constantly c...,3.3,Peraton\r\n3.3,"Herndon, VA",1001 to 5000 employees,2017,Company - Private,Aerospace & Defense,Aerospace & Defense,$1 to $2 billion (USD),-1,False,Chantilly,VA,3.3,3.0
748,748,Salesforce Analytics Consultant,66.5,"Wave6, an Emtec company, innovates with cuttin...",3.9,"Emtec, Inc.\r\n3.9","Jacksonville, FL",501 to 1000 employees,1995,Company - Private,Enterprise Software & Network Solutions,Information Technology,$100 to $500 million (USD),-1,False,Chicago,IL,3.9,25.0
749,749,Managing Data Scientist/ML Engineer,107.5,Managing Data Scientist/ML Engineer\r\n\r\nApp...,3.4,PA Consulting\r\n3.4,"London, United Kingdom",1001 to 5000 employees,1943,Company - Private,Consulting,Business Services,$100 to $500 million (USD),"McKinsey & Company, Accenture, Deloitte",False,Boston,MA,3.4,77.0


In [46]:
df["company_name"] = df.apply(lambda x: x["company name"] if x["rating"]<0 else x["company name"][:-3], axis=1)

In [47]:
df["hourly_binary"] = df.apply(lambda x: 1 if x["hourly"]==True else 0, axis=1)

In [48]:
df

,unnamed: 0,job title,salary estimate,job description,rating,company name,headquarters,size,founded,type of ownership,...,sector,revenue,competitors,hourly,city,state,parsed_num,years_old,company_name,hourly_binary
0,0,Data Scientist,72.0,"Data Scientist\r\nLocation: Albuquerque, NM\r\...",3.8,Tecolote Research\r\n3.8,"Goleta, CA",501 to 1000 employees,1973,Company - Private,...,Aerospace & Defense,$50 to $100 million (USD),-1,False,Albuquerque,NM,3.8,47.0,Tecolote Research\r\n,0
1,1,Healthcare Data Scientist,87.5,What You Will Do:\r\n\r\nI. General Summary\r\...,3.4,University of Maryland Medical System\r\n3.4,"Baltimore, MD",10000+ employees,1984,Other Organization,...,Health Care,$2 to $5 billion (USD),-1,False,Linthicum,MD,3.4,36.0,University of Maryland Medical System\r\n,0
2,2,Data Scientist,85.0,"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\r\n4.8,"Clearwater, FL",501 to 1000 employees,2010,Company - Private,...,Business Services,$100 to $500 million (USD),-1,False,Clearwater,FL,"4, 4.8",10.0,KnowBe4\r\n,0
3,3,Data Scientist,76.5,*Organization and Job ID**\r\nJob ID: 310709\r...,3.8,PNNL\r\n3.8,"Richland, WA",1001 to 5000 employees,1965,Government,...,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",False,Richland,WA,3.8,55.0,PNNL\r\n,0
4,4,Data Scientist,114.5,Data Scientist\r\nAffinity Solutions / Marketi...,2.9,Affinity Solutions\r\n2.9,"New York, NY",51 to 200 employees,1998,Company - Private,...,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",False,New York,NY,2.9,22.0,Affinity Solutions\r\n,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,746,"Technology-Minded, Data Professional Opportuni...",70.5,"At VU, we help Veterans buy homes. To make tha...",4.7,Veterans United Home Loans\r\n4.7,"Columbia, MO",1001 to 5000 employees,2002,Company - Private,...,Finance,Unknown / Non-Applicable,-1,False,Columbia,MO,4.7,18.0,Veterans United Home Loans\r\n,0
747,747,Big Data Engineer,118.5,Overview\r\n\r\n\r\nTechnology is constantly c...,3.3,Peraton\r\n3.3,"Herndon, VA",1001 to 5000 employees,2017,Company - Private,...,Aerospace & Defense,$1 to $2 billion (USD),-1,False,Chantilly,VA,3.3,3.0,Peraton\r\n,0
748,748,Salesforce Analytics Consultant,66.5,"Wave6, an Emtec company, innovates with cuttin...",3.9,"Emtec, Inc.\r\n3.9","Jacksonville, FL",501 to 1000 employees,1995,Company - Private,...,Information Technology,$100 to $500 million (USD),-1,False,Chicago,IL,3.9,25.0,"Emtec, Inc.\r\n",0
749,749,Managing Data Scientist/ML Engineer,107.5,Managing Data Scientist/ML Engineer\r\n\r\nApp...,3.4,PA Consulting\r\n3.4,"London, United Kingdom",1001 to 5000 employees,1943,Company - Private,...,Business Services,$100 to $500 million (USD),"McKinsey & Company, Accenture, Deloitte",False,Boston,MA,3.4,77.0,PA Consulting\r\n,0


In [49]:
df["python"] = df["job description"].apply(lambda x: 1 if "python" in x.lower() else 0)

In [50]:
df["R studio"] = df["job description"].apply(lambda x: 1 if " R " in x or "r-studio" in x.lower() else 0)

In [51]:
df["spark"] = df["job description"].apply(lambda x: 1 if "spark" in x.lower() else 0)

In [52]:
df["aws"] = df["job description"].apply(lambda x: 1 if "aws" in x.lower() else 0)

In [53]:
df["excell"] = df["job description"].apply(lambda x: 1 if "excell" in x.lower() else 0)

In [54]:
df["SQL"] = df["job description"].apply(lambda x: 1 if "sql" in x.lower() else 0)

In [55]:
df["SQL"].value_counts()

1    385
0    366
Name: SQL, dtype: int64

In [56]:
lista_sql = pd.Series([1 if "SQL" in df["job description"][i] else 0 for i in range(len(df["job description"]))]) # list comprehension with else statement 

In [57]:
lista_sql.value_counts()

1    385
0    366
dtype: int64

In [58]:
df = df.drop("unnamed: 0", axis=1)

In [59]:
df.to_csv("Ken Project Data Cleaned.csv", index=False)

In [60]:
pd.read_csv("Ken Project Data Cleaned.csv")

,job title,salary estimate,job description,rating,company name,headquarters,size,founded,type of ownership,industry,...,parsed_num,years_old,company_name,hourly_binary,python,R studio,spark,aws,excell,SQL
0,Data Scientist,72.0,"Data Scientist\r\nLocation: Albuquerque, NM\r\...",3.8,Tecolote Research\r\n3.8,"Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,...,3.8,47.0,Tecolote Research\r\n,0,1,0,0,0,1,0
1,Healthcare Data Scientist,87.5,What You Will Do:\r\n\r\nI. General Summary\r\...,3.4,University of Maryland Medical System\r\n3.4,"Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,...,3.4,36.0,University of Maryland Medical System\r\n,0,1,0,0,0,0,0
2,Data Scientist,85.0,"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\r\n4.8,"Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,...,"4, 4.8",10.0,KnowBe4\r\n,0,1,1,1,0,0,1
3,Data Scientist,76.5,*Organization and Job ID**\r\nJob ID: 310709\r...,3.8,PNNL\r\n3.8,"Richland, WA",1001 to 5000 employees,1965,Government,Energy,...,3.8,55.0,PNNL\r\n,0,1,0,0,0,0,0
4,Data Scientist,114.5,Data Scientist\r\nAffinity Solutions / Marketi...,2.9,Affinity Solutions\r\n2.9,"New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,...,2.9,22.0,Affinity Solutions\r\n,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,"Technology-Minded, Data Professional Opportuni...",70.5,"At VU, we help Veterans buy homes. To make tha...",4.7,Veterans United Home Loans\r\n4.7,"Columbia, MO",1001 to 5000 employees,2002,Company - Private,Lending,...,4.7,18.0,Veterans United Home Loans\r\n,0,0,0,1,0,0,1
747,Big Data Engineer,118.5,Overview\r\n\r\n\r\nTechnology is constantly c...,3.3,Peraton\r\n3.3,"Herndon, VA",1001 to 5000 employees,2017,Company - Private,Aerospace & Defense,...,3.3,3.0,Peraton\r\n,0,1,0,1,0,0,1
748,Salesforce Analytics Consultant,66.5,"Wave6, an Emtec company, innovates with cuttin...",3.9,"Emtec, Inc.\r\n3.9","Jacksonville, FL",501 to 1000 employees,1995,Company - Private,Enterprise Software & Network Solutions,...,3.9,25.0,"Emtec, Inc.\r\n",0,1,0,0,0,0,1
749,Managing Data Scientist/ML Engineer,107.5,Managing Data Scientist/ML Engineer\r\n\r\nApp...,3.4,PA Consulting\r\n3.4,"London, United Kingdom",1001 to 5000 employees,1943,Company - Private,Consulting,...,3.4,77.0,PA Consulting\r\n,0,0,1,1,0,1,1
